In [ ]:
# First we import the module and start up the Matlab interpreter

import os
import numpy as np
from pace_neutrons import Matlab
m = Matlab()


# Simulation and Fitting

For fitting and simulating, Horace makes a difference between the *(Q,E)* - that is `[h,k,l,E]` - and "plot" coordinate system (defined by the projection `[u,v,w,E]` vectors. You can fit a given `sqw` object with *either* a function defined by the `[h,k,l,E]` coordinates *or* the `[u,v,w,E]` coordinates, but you have to use different Horace functions. You can only fit `dnd` objects with functions defined on the plot coordinates `[u,v,w,E]`.

Generally, you fit peak functions (Gaussian, Lorentzians, etc) using the plot coordinates - this means that a function fitting a 1D cut will accept only one dependent coordinate; a function fitting a 2D slice will require 2 input coordinates etc. 

On the other hand, spin wave or phonon models are defined in terms of the (Q,E) coordinates, and you usually have to give all four `[h,k,l,E]` coordinates regardless of the dimension of the cut because they are evaluated on the pixels contributing to each bin.

We will start with showing a simulation of a simple peak (Gaussian) over a cut, and then show a S(Q,w) model to give you a flavour of the syntax.

## Simulation 

We start by going back to some 1D cuts we did earlier this morning, along the [HH0] direction at different energy transfer.

(Aside, why would we want to do constant energy cuts along Q instead of constant Q cuts along energy?)

In [ ]:
sqw_file = '/mnt/ceph-training/course_materials/sqw_files/iron.sqw'
proj = m.line_proj([1,1,0], [-1,1,0], 'type', 'rrr')

# Make an array of 1d cuts:
energy_range = range(80, 161, 20)
my_cuts = m.sqw.empty()
for i, en in enumerate(energy_range):
    my_cuts[i] = m.cut_sqw(sqw_file, proj, [-3,0.05,3], [-1.1,-0.9], [-0.1,0.1], [-10+en, 10+en])

# Plots the cuts using a vectorised operation, offsetting each cut by 0.3 units in y
m.acolor(['black', 'red', 'blue', 'green', 'yellow'])
m.amark(['+', 'o', '*', '.', 'x', 's'], 6)
my_cut_offset = my_cuts + np.arange(0, 1.3, 0.3)
m.dl(my_cut_offset)

We now simulate a set of peak functions to the first cut, with parameters judged "by eye".

We will be using the `mgauss` function to fit, and it takes `3*n` parameters where `n` is the number of peaks, and the parameters are `[area, centre, width]` in that order repeated.

How many peaks are there? What are the approximate values of their centres?

Use the `func_eval` function ([documentation here](https://pace-neutrons.github.io/Horace/unstable/manual/Simulation.html)) to evaluate the Gaussians on the data coordinates of the cut and plot it.

In [ ]:
pars_in = [?, ?, ?,
           ?, ?, ?,
          ???]

mgauss = m.eval('@mgauss')   # We need to do this to get a Matlab function handle to pass to func_eval
peaks_sim = m.func_eval(my_cuts[0], mgauss, pars_in)

m.acolor('black')
m.plot(my_cuts[0])
m.acolor('red')
# Note the peaks simulation doesn't have a background - we add a flat background to match the data
m.pl(peaks_sim + 0.3)

Next we run a simulation of a S(q,w) model function - in fact a ferromagnetic spin wave model which should be able to fit the spinwaves in iron. It is described by the following equations:

$$ E_0 = 8J (1 - \cos(\pi h) \cos(\pi k) \cos(\pi l) $$
$$ B_{\mathrm{iso}} = \frac{E}{1 - \exp\left(- \frac{1.602 E}{T}\right)} $$
$$ S(\mathbf{Q}, E) = \frac{A_0 B_{\mathrm{iso}}}{\pi} \frac{4 \Gamma E_0 }{(E^2 - E_0^2)^2 + 4(\Gamma E)^2} $$

Where $A$ is an amplitude scale factor, $T$ is the temperature in Kelvin $J$ is a ferromagnetic exchange constant in meV, $\Gamma$ is an energy broadening term in meV.

The $E_0$ term is the dispersion, and $\mathbf{Q}=(h, k, l)$.

You should define the $S(\mathbf{Q},E)$ as a Python function which takes $h, k, l, E$ as coordinate inputs as well as the parameters $J, \Delta, \Gamma, A_0$ (the sample temperature $T$ is fixed at 10K for our measurements). The parameters should be given as a list / vector of values `p`.

Then use the `sqw_eval` function ([documentation here](https://pace-neutrons.github.io/Horace/unstable/manual/Simulation.html#sqw-eval)) to evaluate it on the cut.

In [ ]:
def fm_sqw(h, k, l, E, p):
    J = p[0]
    Gamma = p[1]
    A0 = p[2]
    E0 = ??
    Biso = ??
    S = ??
    return S

In [ ]:
J = ?
Gamma = ?
A0 = ?

w_cal = m.sqw_eval(my_cuts[0], fm_sqw, [J, Gamma, A0])
# Plots the data and model together
m.plot(my_cuts[0])
m.pl(w_cal + 0.3)

Explore with different values of the parameters and see if you can get close to the data.

Start by trying to match the amplitude $A_0$ (put any value for $J$ and $\Gamma$. Then try to match $\Gamma$ and then finally $J$ to see if you can reproduce the $Q$ spacing of the peaks.

What do you notice about the amplitude of the data vs that of the calculation?

----

Finally to end the simulations section, we will make a dispersion plot along a line of Q points, using the FM spin wave definition above, using the `disp2sqw_plot` function ([documentation here](https://pace-neutrons.github.io/Horace/unstable/manual/Simulation.html#disp2sqw-plot)).

This function takes a slightly different input from `sqw_eval` - it expects both the intensity $S$ and the dispersion relation $E_0$ as a function of $Q$. In this case, we should return the $S(\mathbf{Q},E=E_0)$ (the intensity at $E=E_0$) value.

In [ ]:
def fm_disp(h, k, l, p):
    J = p[0]
    Gamma = p[1]
    A0 = p[2]
    E0 = ??
    E = E0
    Biso = ??
    S = ??
    return E0, S

In [ ]:
alatt = [2.87, 2.87, 2.87]
angdeg = [90, 90, 90]
lattice = alatt + angdeg

# Reciprocal lattice points to draw dispersion between:
rlp = [[0,0,0], [0,0,1], [0,0,0], [1,0,0], [0,0,0], [1,1,0], [0,0,0], [1,1,1]]
# Input parameters
pars = [35, 30, 80]
# Energy grid
ecent = [0, 0.1, 200];
# Energy broadening term
fwhh = 10

m.disp2sqw_plot(lattice, rlp, fm_disp, pars, ecent, fwhh)
m.lz(0,0.1)

Finally... above notice that the simulated amplitude is equal for all peaks in the 1D cut, but in the data, as Q increases the peak height falls - this is because of the magnetic form factor of iron, which has the following form:

```
# Parameters for the form factor of Fe2+
A=0.0706; a=35.008;  B=0.3589; b=15.358;  C=0.5819; c=5.561;  D=-0.0114
q2 = ((1/(2*2.87))**2) * (h**2 + k**2 + l**2)
ff = A * np.exp(-a*q2) + B * np.exp(-b*q2) + C * np.exp(-c*q2) + D
```

Add a prefactor of `ff**2` to your expression for $S(\mathbf{Q},E)$ and replot and see if you get better agreement in the above.

In [ ]:
# Parameters for the form factor of Fe2+
A=0.0706; a=35.008;  B=0.3589; b=15.358;  C=0.5819; c=5.561;  D=-0.0114

# Define the Python function
def fm_sqw(h, k, l, E, p):
    J = p[0]
    Gamma = p[1]
    A0 = p[2]
    E0 = ??
    Biso = ??
    q2 = ((1/(2*2.87))**2) * (h**2 + k**2 + l**2)
    ff = A * np.exp(-a*q2) + B * np.exp(-b*q2) + C * np.exp(-c*q2) + D
    return (ff**2) * ??

J = ??      # Exchange parameter in meV
Gamma = ??  # Intrinsic linewidth in meV (inversely proportional to excitation lifetime)
A0 = ??     # Magnitude of the intensity of the excitation (arbitrary units)

w_cal = m.sqw_eval(my_cuts[0], fm_sqw, [J, Gamma, A0])
# Plots the data and model together
m.plot(my_cuts[0])
m.pl(w_cal + 0.3)

## Fitting a single 1D cut with some peaks

We now try to fit a set of Gaussian functions to the peaks in the first cut of the set of cuts we made at the start of the session

Look at first curve - how many peaks are there? We will be using the `mgauss` function to fit, and it takes `3*n` parameters where `n` is the number of peaks, and the parameters are `[area, centre, width]` in that order repeated.

It does not accept a background offset, so first subtract a constant background from the first cut, then use the `multifit` class to fit it.

In [ ]:
%matlab_plot_mode inline

pars_in = [?, ?, ?,  # area, centre, width of first peak
           ?, ?, ?,
           ???]

mfobj = m.multifit_func(my_cuts[0] - 0.3)
mgauss = m.eval('@mgauss')  # We need to do this to get a Matlab function handle to pass to multifit
mfobj = mfobj.set_fun(mgauss, pars_in)
wfit, fitdata = mfobj.fit()

m.acolor('black')
m.plot(my_cuts[0] - 0.3)
m.acolor('red')
m.pl(wfit)

print('Fitted parameters are:')
fitpars = np.squeeze(fitdata['p'])
fiterrs = np.squeeze(fitdata['sig'])
for ii in range(int(len(pars_in)/3)):
    print(f"{fitpars[ii*3]:0.3f}({fiterrs[ii*3]:0.3f}) {fitpars[ii*3+1]:0.3f}({fiterrs[ii*3+1]:0.3f}) {fitpars[ii*3+2]:0.3f}({fiterrs[ii*3+2]:0.3f})")


The `fitdata` structure/dictionary has a lot of information about the fit, including the covariance matrix. Let's print it out and see. 

In [ ]:
np.set_printoptions(linewidth=140, precision=3, suppress=True)
print(fitdata['corr'])

How can we improve this fit? (Hint: which variables have high covariance, indicating that they are correlated / not independent - consider fixing some of them)

In [ ]:
# Run fit again, but fixing some parameters. 
# We specify this as a vector of boolean values indicating which parameters are *free* to vary

pars_in = ??
pars_free = ??

mfobj = m.multifit_func(my_cuts[0] - 0.3)
mgauss = m.eval('@mgauss')  # We need to do this to get a Matlab function handle to pass to multifit
mfobj = mfobj.set_fun(mgauss, pars_in)
mfobj = mfobj.set_free(pars_free)
wfit, fitdata = mfobj.fit()

m.acolor('black')
m.plot(my_cuts[0] - 0.3)
m.acolor('red')
m.pl(wfit)

print('Fitted parameters are:')
fitpars = np.squeeze(fitdata['p'])
fiterrs = np.squeeze(fitdata['sig'])
for ii in range(int(len(pars_in)/3)):
    print(f"{fitpars[ii*3]:0.3f}({fiterrs[ii*3]:0.3f}) {fitpars[ii*3+1]:0.3f}({fiterrs[ii*3+1]:0.3f}) {fitpars[ii*3+2]:0.3f}({fiterrs[ii*3+2]:0.3f})")


It's better but still not so good - there are still too many parameters.

Notice that the peak positions should be symmetric about Q=0 - we can thus fix the centre positions of those peaks to be negative of each other. We can do this using "parameter bindings", [described here in the documentation](https://pace-neutrons.github.io/Horace/unstable/manual/Multifit.html#binding-parameters).

The basic syntax is that each binding is a set of three values in a cell array, denoting:

```
(index_bound_par, index_free_par, ratio)
```

So, say we put the first peak around 0.9 and the second peak around -0.9, the peak position is thus parameter index 2 and 5 (in Matlab 1-based indexing because this is refered to by Horace code which is in Matlab). We would thus use:

```
(2, 5, -1)
```

To specify then bindings.

Now try to fit with binding the positive and negative peak centres in Q.

(To set multiple bindinds, used a nested tuple in Python [nested cell-arrays in Matlab], e.g. `((2, 5, -1), (8, 11, -1))`)

In [ ]:
# Run fit again, but fixing some parameters. 
# We specify this as a vector of boolean values indicating which parameters are *free* to vary

pars_in = ??
pars_free = ??
pars_bind = ??

mfobj = m.multifit_func(my_cuts[0] - 0.3)
mgauss = m.eval('@mgauss')  # We need to do this to get a Matlab function handle to pass to multifit
mfobj = mfobj.set_fun(mgauss, pars_in)
mfobj = mfobj.set_free(pars_free)
mfobj = mfobj.set_bind(pars_bind)
wfit, fitdata = mfobj.fit()

m.acolor('black')
m.plot(my_cuts[0] - 0.3)
m.acolor('red')
m.pl(wfit)

print('Fitted parameters are:')
fitpars = np.squeeze(fitdata['p'])
fiterrs = np.squeeze(fitdata['sig'])
for ii in range(int(len(pars_in)/3)):
    print(f"{fitpars[ii*3]:0.3f}({fiterrs[ii*3]:0.3f}) {fitpars[ii*3+1]:0.3f}({fiterrs[ii*3+1]:0.3f}) {fitpars[ii*3+2]:0.3f}({fiterrs[ii*3+2]:0.3f})")


Finally, we can also set a background function, rather than just subtracting a constant from the data before fitting. [The documentation is here](https://pace-neutrons.github.io/Horace/unstable/manual/Multifit.html#background-functions).

We will use the `linear_bg` function which takes two parameters `[yintercept, slope]`. Use a zero slope and `0.3` as the y-intercept to start.

In [ ]:
# Run fit again, but fixing some parameters. 
# We specify this as a vector of boolean values indicating which parameters are *free* to vary

pars_in = ??
pars_free = ??

mfobj = m.multifit_func(my_cuts[0])
mgauss = m.eval('@mgauss')  # We need to do this to get a Matlab function handle to pass to multifit
mfobj = mfobj.set_fun(mgauss, pars_in)
mfobj = mfobj.set_free(pars_free)
# Define a linear background
linear_bg = m.eval('@linear_bg')
mfobj = mfobj.set_bfun(linear_bg, [0.3, 0])
wfit, fitdata = mfobj.fit()

m.acolor('black')
m.plot(my_cuts[0])
m.acolor('red')
m.pl(wfit)

print('Fitted parameters are:')
fitpars = np.squeeze(fitdata['p'])
fiterrs = np.squeeze(fitdata['sig'])
for ii in range(int(len(pars_in)/3)):
    print(f"{fitpars[ii*3]:0.3f}({fiterrs[ii*3]:0.3f}) {fitpars[ii*3+1]:0.3f}({fiterrs[ii*3+1]:0.3f}) {fitpars[ii*3+2]:0.3f}({fiterrs[ii*3+2]:0.3f})")


## Fitting using a S(Q,w) model

We now try to fit the FM spin wave model function we defined above.

Everything is almost the same as with fitting Gaussians, except you should use `multifit_sqw` instead of `multifit_func` - the suffix denoting that the fit function in this case takes the 4 coordinates `[h,k,l,E]` instead of the plot coordinates as input.

Otherwise the syntax is almost identical - set up a fit (with a linear background function) and try to see if you can fit the data. (You should use the function you defined _including_ the form factor).

In [ ]:
# Parameters for the form factor of Fe2+
A=0.0706; a=35.008;  B=0.3589; b=15.358;  C=0.5819; c=5.561;  D=-0.0114

# Define a linear background
linear_bg = m.eval('@linear_bg')

# Define the Python function
def fm_sqw(h, k, l, E, p):
    J = p[0]
    Gamma = p[1]
    A0 = p[2]
    E0 = ??
    Biso = ??
    q2 = ((1/(2*2.87))**2) * (h**2 + k**2 + l**2)
    ff = A * np.exp(-a*q2) + B * np.exp(-b*q2) + C * np.exp(-c*q2) + D
    return (ff**2) * ??

J = ??      # Exchange parameter in meV
Gamma = ??  # Intrinsic linewidth in meV (inversely proportional to excitation lifetime)
A0 = ??     # Magnitude of the intensity of the excitation (arbitrary units)

pars_in = [J, Gamma, A0]

mfobj = m.multifit_sqw(my_cuts[0])
mfobj = mfobj.set_fun(fm_sqw, pars_in)
mfobj = mfobj.set_bfun(linear_bg, [0.3, 0])
wfit, fitdata = mfobj.fit()

m.acolor('black')
m.plot(my_cuts[0])
m.acolor('red')
m.pl(wfit)

print(fitdata)

If you're satisfied with fitting the first cut there, now try to fit all the cuts (at different energies) we made at the start.

This can be done simply by give the full list of cuts to `multifit_sqw` when you construct it instead of just the first cut. Everything else will stay the same.

In [ ]:
mfobj = m.multifit_sqw(my_cuts)
mfobj = mfobj.set_fun(fm_sqw, pars_in)
mfobj = mfobj.set_bfun(linear_bg, [0.3, 0])
wfit, fitdata = mfobj.fit()

%matlab_fig -h 800 -w 600 -r 100

# Plots the cuts using a vectorised operation, offsetting each cut by 0.5 units in y
m.acolor(['black', 'red', 'blue', 'green', 'yellow'])
m.amark(['+', 'o', '*', '.', 'x', 's'], 6)
my_cut_offset = my_cuts + np.arange(0, 2.1, 0.5)
my_fit_offset = wfit + np.arange(0, 2.1, 0.5)
m.dp(my_cut_offset)
m.pl(my_fit_offset)

print(fitdata)

## Resolution convolution

Finally we add resolution convolution calculation to the fit. This is almost exactly the same as using `multifit_sqw` except that we now use the `tobyfit` class instead and we have to define some instrument and sample parameters. [The documentation is here](https://pace-neutrons.github.io/Horace/unstable/user_guide/Resolution_convolution.html#the-tobyfit-class)

In [ ]:
# Calculate spectra with full instrument resolution convolution

# Define sample parameters
xgeom = [0,0,1]
ygeom = [0,1,0]
shape = 'cuboid'
shape_pars = [0.01,0.05,0.01]

# Define instrument parameters
ei = 400    # Incident energy in meV
freq = 600  # Chopper frequency in Hz
chop = 's'  # Chopper rotor identifier (ask instrument scientist)

# Sets the sample and instrument on the workspace
my_cuts[0] = my_cuts[0].set_sample(m.IX_sample(xgeom, ygeom, shape, shape_pars))
my_cuts[0] = my_cuts[0].set_instrument(m.merlin_instrument(ei, freq, chop))

tbfobj = m.tobyfit(my_cuts[0])
tbfobj = tbfobj.set_fun(fm_sqw, fitdata['p'])  # Use previous best fit parameters
tbfobj = tbfobj.set_bfun(linear_bg, fitdata['bp'])
w_tbf = tbfobj.simulate()

m.acolor('black')
m.plot(my_cuts[0])
m.acolor('red')
m.pl(w_tbf)